### I have Try to download from Azure databricks Ipython Notebook but based on the size of the file i rewrite in VS with explaining everything 

In [ ]:
%pip install toml

# Toml 
- here we use toml that to read configuration data from a TOML file. and we can use a different way in azure or also we can use dotenv library that will read from .env files 

In [ ]:
import toml

with open ("/Workspace/Users/xavei.15-6@hotmail.com/info.toml","r") as f:
    config_data = toml.load(f)

# mounting and unmounting 
- First we unmount the storage locatio before that to avoid a error and it's not necessary to do that

In [ ]:
dbutils.fs.unmount("/mnt/sof-mountpoint")

# why of mounting Azure Blob storage 
- mounting an Azure Blob Storage container on Databricks File System (DBFS). For : \
       - storageAccountName \
       - containerName \
       - applicationId \
       - directoryID \
       - secretValue 
- we extract information from the `config_data` dictionary. This dictionary likely comes from a configuration file that stores sensitive details securely.
- For `endpoint` it's acctuly mean that we constructs the endpoint URL for OAuth token acquisition using the directoryID. This URL points to the AAD service to get authorization tokens.
- for `Source` we want to mount the Blob Storage contianer So we use the URI with format `abfss://` File system and that Follow the contianer name and storage account and end with the domain. 
- for configuration we use the key set for authentication type which we use `OAuth` for authorization , then we specified the key `OAuth` provider which is : `org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider` , then we set the application id and client secret and token endpoint



In [ ]:
storageAccountName = config_data['storageAccountName']
containerName = config_data['containerName']
applicationId = config_data['applicationId']
directoryID = config_data['directoryID']
secretValue = config_data['secretValue']
endpoint = 'https://login.microsoftonline.com/' + directoryID + '/oauth2/token'
source = 'abfss://' + containerName + '@' + storageAccountName + '.dfs.core.windows.net/'
mountPoint = "/mnt/sof-mountpoint"

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": applicationId,
           "fs.azure.account.oauth2.client.secret": secretValue,
           "fs.azure.account.oauth2.client.endpoint": endpoint}


dbutils.fs.mount(source = source,mount_point = mountPoint, extra_configs = configs)

In [ ]:
#display(dbutils.fs.ls('/mnt/sof-mountpoint')) # /mnt/sof-mountpoint -> it's will show us the two files that we have it in our Storage account and inside the contianer that we have, we have two folders one for daily and weekly loaded files and archive old file .


display(dbutils.fs.ls('/mnt/sof-mountpoint/Landing'))# if we want to see what inside Landing folder we can add it into the 

# Creating a spark session
is we use the databricks inside azure here we don't need to create a session or sparkcontext, but if you work localy you need to createa a session before you start.

from pyspark.sql import SparkSession

spark = (SparkSession
         .builder
         .appName("Table Loading")
         .getOrCreate())

sc = spark.sparkContext

# Listing the Mounting Location Storage
- By using the utilities function that we can review the blob storage and we direct it into `Sample-data-training`

In [ ]:
display(dbutils.fs.ls('/mnt/sof-mountpoint/Landing/sample-data-training/'))

In [ ]:
# now we need to create a dataframe for Post and to doing that we need to use file location which mean that we recall the mount point of storage workshop that data mount in /mnt/sof-mountpoint/Landing
file_location = '/mnt/sof-mountpoint/Landing/sample-data-training/post/*'# * here mean all file inside the path 

posts = spark.read.parquet(file_location) # here we creating the DataFrame be read the file_location as parquet which the files format inside the mount point 
num_columns = len(posts.schema.fields) # info about the num of columns that we have 
print(f'the number of columns that we have :{num_columns}') # 17

display(posts) # 2,066 rows | 9.21 seconds runtime

# Schemas and DataFrames
- There are several benefit from creating DataFrame and Save into local file system:
> - Side of Scalability : we can store and process massive datasets without worrying about infrastructure limitations. which databricks leverages the underlying Azure cloud storage, which offers vast scalability. 
> - Side of Cost-Effectiveness: provides flexible storage options that cater to different needs and budgets 
> - Side of Performance and Efficiency: is readily accessible for various analytics tasks. Leveraging distributed processing, Databricks can analyze large datasets efficiently.Optimized data formats like Parquet (used in your code) further enhance query performance by enabling columnar storage and data skipping. 
> - Side of version control and Collabration : when the data saved in Databricks workspaces is accessible to authorized users, facilitating collaborative data analysis and exploration.Databricks provides version control capabilities, allowing you to track changes, revert to previous versions, and ensure data integrity. 
> - Side of integration and security : Azure Databricks seamlessly integrates with other Azure services like Azure Machine Learning, Azure Data Factory, and Azure Synapse Analytics. This streamlines data pipelines and workflows. Databricks adheres to stringent security standards and offers various access control mechanisms to protect sensitive data.

In [ ]:
# in PostType and user we need to define(Build) the structure of the dataframe

from pyspark.sql.types import *

pt_schema = StructType([
    StructField("id", IntegerType(),True),
    StructField("Type",StringType(),True)
])

In [ ]:
# now we creating the PostType dataframe by defining the File_location and then we create it 
pt_file_location = '/mnt/sof-mountpoint/Landing/sample-data-training/PostTypes.txt'

post_type = spark.read.option('header',True).option('sep',',').schema(pt_schema).csv(pt_file_location)
display(post_type)

In [ ]:
# in PostType and user we need to define(Build) the structure of the dataframe

users_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("Age", IntegerType(), True),
    StructField("CreationDate", DateType(), True),
    StructField("DisplayName", StringType(), True),
    StructField("DownVotes", IntegerType(), True),
    StructField("EmailHash", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("Reputation", IntegerType(), True),
    StructField("UpVotes", IntegerType(), True),
    StructField("Views", IntegerType(), True),
    StructField("WebsiteUrl", StringType(), True),
    StructField("AccountId", IntegerType(), True)
])

In [ ]:
# now we creating the user dataframe by defining the File_location and then we create it 
user_file_location = '/mnt/sof-mountpoint/Landing/sample-data-training/users.csv'
user = spark.read.option('header',True).option('sep',',').schema(users_schema).csv(user_file_location)

display(user)

In [ ]:
# Saving the dataframes for easy retrieval So we will save 3 tables to databricks local file system
posts.write.parquet('/temp/projects/sof/post.parquet')
post_type.write.parquet('/temp/projects/sof/post_type.parquet')
user.write.parquet('/temp/projects/sof/user.parquet')

- If we didn't set the overwrite mode may will get the below error that mean we already have these tables 
> [PATH_ALREADY_EXISTS] Path dbfs:/temp/projects/sof/post.parquet already exists. Set mode as "overwrite" to overwrite the existing path. SQLSTATE: 42K04
File .

In [ ]:
# review the local file system
display(dbutils.fs.ls("/temp/projects/sof/"))

# Join tables and filter data

In [ ]:
# First we need to import the necessary library
from pyspark.sql.functions import split, translate, trim, explode, regexp_replace, col, lower

In [ ]:
# now we want to read the tables
posts= spark.read.parquet('/temp/projects/sof/post.parquet')
post_type = spark.read.parquet('/temp/projects/sof/post_type.parquet')
user = spark.read.parquet('/temp/projects/sof/user.parquet')

In [ ]:
# at this moment, we only use Posts and posttypes to train the model. so let's join them with the posttype id sence that the Post have column called posttypeid where we can 
# can join the two table through this key
join_p_pt = posts.join(post_type,posts.PostTypeId == post_type.id)
display(join_p_pt)

# Filter the data
In the `posttypes` table, there is a column called `Type` which indicates if the posts is a question or an answer. We only need the `question` entires. For these `Question` rows, we will run machine learning model on the join the `Body` column of the `Posts` table. To tell what topic this post is about.

In [ ]:
filtering_join_p_pt = join_p_pt.filter(col("Type") == "Question")
display(filtering_join_p_pt)

# Here are examples of `BODY AND TAGS` content

In [ ]:
# BODY:<p>I have an abstract class with a protected variable</p> <pre><code>abstract class Beverage { protected string description; } </code></pre> <p>I can't access it from a subclass. 
    # Intellisense doesn't show it accessible. Why is that so?</p> <pre><code>class Espresso:Beverage { //this.description ?? }
    # </code></pre>

# Tags: <c#><.net><abstract-class><protected>

# Formatting for Machine Learning:
- Machine learning algorithms typically work best with numerical data. Text data like the content in your Body and Tags columns needs preprocessing to be suitable for training these models.Removing HTML Tags, and we can use regexp_replace function removes HTML tags, and for Tags columns we can use split

In [ ]:
# we need now to Formatting the 'Body' and `Tag` columns for machine learning training
df = (filtering_join_p_pt.withColumn('Body',\
       regexp_replace(filtering_join_p_pt.Body, r'<.*?>', ''))# Transforming HTML code to strings
      
      .withColumn("Tags", split(trim(translate(col("Tags"), "<>", " ")), " ")) # Making a list of the tags
)

display(df)

In [ ]:
# Filter the dataframe to only include questions
df = df.filter(col("Type") == "Question")
display(df)

In [ ]:
df = df.select(col("Body").alias("text"), col("Tags"))

In [ ]:
# Producing the tags as individual tags instead of an array
# This is duplicating the posts for each possible tag
df = df.select("text", explode("Tags").alias("tags"))
display(df)

- If we didn't set the overwrite mode may will get the below error that mean we already have these the Checkpoint 
> Path dbfs:/tmp/project/sof.df.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

# Checkpoint 
- as we know ML and data processing tasks can take hours or even days, unexpected events like cluster crashes, network issues, or software errors can interrupt the process and lead to wasted compute resources and time and that lead to `fault tolerance`
- during the computation the checkpoint like saving point if the cluster terminated then you can start the process from last save checkpoint.

In [ ]:
# saving the file as a checkpoint (in case the cluster gets terminated)

df.write.parquet("/tmp/project/sof.df.parquet")

### df.cache():
> This method instructs Spark to cache the DataFrame df in memory across the cluster.Purpose of using `cache()` Caching improves the performance of subsequent operations on the same DataFrame because Spark doesn't need to re-read the data from the original source (e.g., disk storage) every time.
- Benefits:
  - Faster execution for repeated operations on the same DataFrame.
  - Useful when working with large datasets that are accessed multiple times within your notebook.
> so we cache the DataFrame df after applying transformations like explode. This ensures faster execution if you perform further operations on the same DataFrame within your notebook.
### df.count():
- we use the `count` After caching fro serveal purpose:
  - Verifies if the data has been loaded successfully and provides the total number of rows after the explode operation.
  - Since the DataFrame is cached, counting should be relatively fast.

In [ ]:
df.cache()
df.count()

# Text Cleaning Preprocessing
- pyspark.sql.functions.regexp_replace is used to process the text
* by using the regexp_replace we will do:
   - Remove URLs such as http://stackoverflow.com
   - Remove special characters
   - Substituting multiple spaces with single space
   - Lowercase all text
   - Trim the leading/trailing whitespaces

In [ ]:
from pyspark.sql.functions import regexp_replace

In [ ]:
data_cleaning = df.withColumn('text',regexp_replace("text",r"http\S+"," ")) \
            .withColumn('text',regexp_replace("text",r"[^a-zA-z]"," ")) \
            .withColumn('text',regexp_replace("text",r"\s+"," ")) \
            .withColumn('text',lower('text')) \
            .withColumn('text',trim('text'))
display(data_cleaning)

# Machine Learning Model Training

In [ ]:
# Importing all the libraries
from pyspark.ml.feature import Tokenizer 
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import split, translate, trim, explode, regexp_replace, col, lower
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#### for train and test:
- we calls randomSplit() function which give us a split into two new dataframe based on giving weights
    * `[0.9,0.1]` which we set for training `%90` and for testing `%10`.
    * to ensure that whenever we run this line of code and the data will spilt in same way we use `seed`

In [ ]:
# Train Test Split
train, test = data_cleaning.randomSplit([0.9, 0.1], seed=20200819)

# Tokenizer 
- to handle converting textual data into a format suitable for machine learning models we will: 
    * takes raw text data as input and transforms it into a sequence of tokens, which are smaller meaningful units like words or sub-words.
- and Tokenization will help to removing punctuation and stop words (common words like "the", "a", "an") that don't contribute much to the meaning.


In [ ]:
# Feature Transformer Tokenizer
tokenizer_field = Tokenizer(inputCol="text", outputCol="tokens")
tokenized = tokenizer_field.transform(data_cleaning)
display(tokenized)

In [ ]:
# Stopword Removal
sw_remover = StopWordsRemover(inputCol='tokens',outputCol='filtered_word')
sw = sw_remover.transform(tokenized)
display(sw)


- count vectorization it's aims to convert textual data into a numerical representation suitable for machine learning algorithms, also create a vocabulary (dictionary) of all unique words encountered in the training data. This vocabulary defines the features for the resulting numerical representation.


In [ ]:
# CountVectorizer (TF - Term Frequency)
count_vectorizer = CountVectorizer(vocabSize=2**16,inputCol='filtered_word',outputCol='vectorization_count')
cv_model=count_vectorizer.fit(sw)
cv_text = cv_model.transform(sw)
display(cv_text)

- as we know for each document (text sample) in your data, the count vectorizer counts the number of times each word from the vocabulary appears in that document. This represents the term frequency (TF) for each word.

In [ ]:
# TF-IDF Vectorization
idf = IDF(inputCol='vectorization_count', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
idf_model = idf.fit(cv_text)
text_idf = idf_model.transform(cv_text)

display(text_idf)

- as we know also ML can't directly process text data like the tags in your text_idf DataFrame so They require numerical features for computations. Label encoding addresses this by assigning a unique integer value to each distinct category (tag) in the "tags" column.

In [ ]:
#Label Encoding

label_encoder = StringIndexer(inputCol = "tags", outputCol = "label")
le_model = label_encoder.fit(text_idf)
final = le_model.transform(text_idf)

display(final)

In [ ]:
# Model Training
logistic_reg = LogisticRegression(maxIter=100)

logistic_reg_model = logistic_reg.fit(final)

predictions = logistic_reg_model.transform(final)
display(predictions)

In [ ]:
# Model Evalution
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
roc_auc = evaluator.evaluate(predictions)
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(predictions.count())

print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))

In [ ]:
# Creating the pipeline
pipeline = Pipeline(stages=[tokenizer_field, sw_remover, count_vectorizer, idf, label_encoder, logistic_reg])


In [ ]:
# Fitting and transforming (predicting) using the pipeline
pipeline_model = pipeline.fit(train)
predictions = pipeline_model.transform(test)

In [ ]:
# Saving model object to the /mnt/deBDProject directory. Yours name may be different.
pipeline_model.save('/mnt/sof-mountpoint/model')

# Save the the String Indexer to decode the encoding. We need it in the future Sentiment Analysis.
le_model.save('/mnt/sof-mountpoint/stringindexer')

In [ ]:
# Review the directory
display(dbutils.fs.ls("/mnt/sof-mountpoint/model"))